In [1]:
%load_ext autoreload
%autoreload 2

### Quickstart

In [3]:
from kogito.models.bart.comet import COMETBART
from kogito.inference import CommonsenseInference

# Load pre-trained model from HuggingFace
model = COMETBART.from_pretrained("mismayil/comet-bart-ai2")

# Initialize inference module with a spacy language pipeline
csi = CommonsenseInference(language="en_core_web_sm")

# Run inference
text = "PersonX becomes a great basketball player"
kgraph = csi.infer(text, model)

# Save output knowledge graph to JSON file
kgraph.to_jsonl("results/kgraph.json")

Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


100%|██████████| 1/1 [00:17<00:00, 17.54s/it]


### Knowledge

In [4]:
from kogito.core.head import KnowledgeHead
from kogito.core.knowledge import Knowledge
from kogito.core.relation import X_NEED

head = KnowledgeHead("PersonX buys lunch")
knowledge = Knowledge(head=head, relation=X_NEED, tails=["bring a wallet"])

### Knowledge Graph

In [5]:
from kogito.core.knowledge import Knowledge, KnowledgeGraph
from kogito.core.head import KnowledgeHead
from kogito.core.relation import X_NEED, CAUSES

knowledge1 = Knowledge(head=KnowledgeHead("PersonX buys lunch"), relation=X_NEED, tails=["bring a wallet"])
knowledge2 = Knowledge(head=KnowledgeHead("Throwing a party"), relation=CAUSES, tails=["have fun"])

kgraph = KnowledgeGraph([knowledge1, knowledge2])

In [6]:
for knowledge in kgraph:
    print(knowledge)

Knowledge(head="PersonX buys lunch", relation="xNeed", tails=['bring a wallet'])
Knowledge(head="Throwing a party", relation="Causes", tails=['have fun'])


In [7]:
# From csv
kgraph1 = KnowledgeGraph.from_csv("sample_graph.tsv", sep="\t", header=None)

# From jsonl (list of json objects)
kgraph2 = KnowledgeGraph.from_jsonl("sample_graph.jsonl", head_attr="source", relation_attr="rel", tails_attr="targets")

In [8]:
# Union
kgraph3 = kgraph1 + kgraph2 # kgraph1.union(kgraph2)

# Intersection
kgraph3 = kgraph1 & kgraph2 # kgraph1.intersection(kgraph2)

# Difference
kgraph3 = kgraph1 - kgraph2 # kgraph1.difference(kgraph2)

In [9]:
kgraph3.to_jsonl("results/sample_graph3.jsonl")

### Saving models

In [ ]:
model.save_pretrained("./comet-bart")

### Customizing processors

In [10]:
csi.processors

{'head': ['sentence_extractor',
  'noun_phrase_extractor',
  'verb_phrase_extractor'],
 'relation': ['simple_relation_matcher', 'graph_relation_matcher']}

In [11]:
csi.remove_processor("noun_phrase_extractor")

In [12]:
csi.processors

{'head': ['sentence_extractor', 'verb_phrase_extractor'],
 'relation': ['simple_relation_matcher', 'graph_relation_matcher']}

### Custom Head Extractor

In [13]:
from typing import Optional, List
from spacy.tokens import Doc
import spacy

from kogito.core.processors.head import KnowledgeHeadExtractor, KnowledgeHead

class AdjectiveHeadExtractor(KnowledgeHeadExtractor):
   def extract(self, text: str, doc: Optional[Doc] = None) -> List[KnowledgeHead]:
      if not doc:
            doc = self.lang(text)

      heads = []

      for token in doc:
            if token.pos_ == "ADJ":
               heads.append(KnowledgeHead(text=token.text, entity=token))

      return heads

adj_extractor = AdjectiveHeadExtractor("adj_extractor", spacy.load("en_core_web_sm"))
csi.add_processor(adj_extractor)

In [14]:
csi.processors

{'head': ['sentence_extractor', 'verb_phrase_extractor', 'adj_extractor'],
 'relation': ['simple_relation_matcher', 'graph_relation_matcher']}

In [15]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph

csi = CommonsenseInference()


### Custom Relation Matcher

In [16]:
from typing import List, Tuple

from kogito.core.processors.head import KnowledgeHead
from kogito.core.processors.relation import KnowledgeRelationMatcher
from kogito.core.relation import KnowledgeRelation, X_NEED, CAUSES

class ConstantRelationMatcher(KnowledgeRelationMatcher):
   def match(
      self, heads: List[KnowledgeHead], relations: List[KnowledgeRelation] = None, **kwargs
   ) -> List[Tuple[KnowledgeHead, KnowledgeRelation]]:
      head_relations = []

      for head in heads:
            head_relations.append((head, X_NEED))
            head_relations.append((head, CAUSES))

      return head_relations

const_rel_matcher = ConstantRelationMatcher("const_rel_matcher", spacy.load("en_core_web_sm"))
csi.add_processor(const_rel_matcher)

In [17]:
csi.processors

{'head': ['sentence_extractor',
  'noun_phrase_extractor',
  'verb_phrase_extractor'],
 'relation': ['simple_relation_matcher',
  'graph_relation_matcher',
  'const_rel_matcher']}

### Dry-run

In [18]:
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True)
kgraph.to_jsonl("results/kgraph_dry_run.json")

Extracting knowledge heads...
Matching knowledge heads with relations...


In [16]:
text = "I wanted to feed him. he didnt listen to me"
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True)
kgraph.to_jsonl("results/kgraph_dry_run_2.json")

Extracting heads...
Matching relations...


### Relation subset

In [19]:
from kogito.core.relation import OBJECT_USE, CAUSES
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True, relations=[OBJECT_USE, CAUSES])
kgraph.to_jsonl("results/kgraph_rel_subset.json")

Extracting knowledge heads...
Matching knowledge heads with relations...


### No Head extraction

In [20]:
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True, extract_heads=False)
kgraph.to_jsonl("results/kgraph_no_head_extract.json")

Matching knowledge heads with relations...


### No Relation matching and no subset of relations

In [21]:
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True, match_relations=False)

Extracting knowledge heads...


### No Relation matching with subset of relations

In [22]:
from kogito.core.relation import DESIRES, CAUSES
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True, match_relations=False, relations=[CAUSES, DESIRES])
kgraph.to_jsonl("results/kgraph_no_match_subset.json")

Extracting knowledge heads...


### No Head extraction, no Relation matching with subset of relations (hence, ultimate manual specification)

In [23]:
from kogito.core.relation import DESIRES, CAUSES
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(text, dry_run=True, extract_heads=False, match_relations=False, relations=[CAUSES, DESIRES])
kgraph.to_jsonl("results/kgraph_manual.json")

### Custom heads

In [24]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph

csi = CommonsenseInference()
text = "Gabby always brought cookies to school."
kgraph: KnowledgeGraph = csi.infer(heads=["post office", "to get out of the room"], dry_run=True)
kgraph.to_jsonl("results/kgraph_manual_heads.json")

Matching knowledge heads with relations...


### Model based Relation matching (SWEM)

In [25]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.core.processors.relation import SWEMRelationMatcher
from kogito.models.bart.comet import COMETBART
from kogito.core.model import KnowledgeModel
import time

csi = CommonsenseInference()
csi.remove_processor("simple_relation_matcher")
swem_matcher = SWEMRelationMatcher("swem_relation_matcher")
csi.add_processor(swem_matcher)
start = time.time()
# model: KnowledgeModel = COMETBART.from_pretrained("mismayil/comet-bart-ai2")
kgraph: KnowledgeGraph = csi.infer(heads=["banana", "love another", "Student gets a card"], dry_run=True)
end = time.time()
print(f"Took {end-start} seconds")
kgraph.to_jsonl("results/kgraph_modelbased_relations_swem.json")

Downloading: 100%|██████████| 204/204 [00:00<00:00, 84.9kB/s]
Downloading: 100%|██████████| 160M/160M [00:10<00:00, 15.5MB/s] 


Matching knowledge heads with relations...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/mismayil/opt/anaconda3/envs/kogito-test/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 191.39it/s]
Took 0.8768618106842041 seconds


/Users/mismayil/opt/anaconda3/envs/kogito-test/lib/python3.8/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


### Model based Relation matching (DistilBERT)

In [26]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.core.processors.relation import DistilBERTRelationMatcher
from kogito.core.model import KnowledgeModel
import time

csi = CommonsenseInference()
csi.remove_processor("simple_relation_matcher")
dbert_matcher = DistilBERTRelationMatcher("dbert_relation_matcher")
csi.add_processor(dbert_matcher)
start = time.time()
# model: KnowledgeModel = COMETBART.from_pretrained("mismayil/comet-bart-ai2")
kgraph: KnowledgeGraph = csi.infer(heads=["banana", "love another", "Student gets a card"], dry_run=True)
end = time.time()
print(f"Took {end-start} seconds")
kgraph.to_jsonl("results/kgraph_modelbased_relations_dbert.json")

Downloading: 100%|██████████| 235/235 [00:00<00:00, 63.7kB/s]
Downloading: 100%|██████████| 265M/265M [00:16<00:00, 16.4MB/s] 
Downloading: 100%|██████████| 483/483 [00:00<00:00, 226kB/s]
Downloading: 100%|██████████| 268M/268M [00:15<00:00, 17.1MB/s] 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Matching knowledge heads with relations...


Downloading: 100%|██████████| 232k/232k [00:00<00:00, 700kB/s] 
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 15.2kB/s]
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  6.77it/s]
Took 3.6173558235168457 seconds


### Model based Relation Matching (BERT)

In [28]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.core.processors.relation import BERTRelationMatcher
from kogito.core.model import KnowledgeModel
import time

csi = CommonsenseInference()
csi.remove_processor("simple_relation_matcher")
bert_matcher = BERTRelationMatcher("dbert_relation_matcher")
csi.add_processor(bert_matcher)
start = time.time()
# model: KnowledgeModel = COMETBART.from_pretrained("mismayil/comet-bart-ai2")
kgraph: KnowledgeGraph = csi.infer(heads=["banana", "love another", "Student gets a card"], dry_run=True)
end = time.time()
print(f"Took {end-start} seconds")
kgraph.to_jsonl("results/kgraph_modelbased_relations_bert.json")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Matching knowledge heads with relations...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s]
Took 2.3340957164764404 seconds


### GPT-3 Based Commonsense Inference

In [29]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.models.gpt3.zeroshot import GPT3Zeroshot
import time, os

csi = CommonsenseInference()
csi.remove_processor("simple_relation_matcher")

model = GPT3Zeroshot(api_key="", model_name="text-davinci-002")
sample_graph = KnowledgeGraph.from_csv("sample_graph.tsv", sep="\t", header=None)
heads = ["PersonX accuses PersonY of cheating", "PersonX aces PersonX's exam"]

start = time.time()
kgraph = csi.infer(model=model, heads=heads, sample_graph=sample_graph, model_args={"debug": True})
end = time.time()

print(f"Took {end-start} seconds")
kgraph.to_jsonl("results/kgraph_gpt3.json")

Matching knowledge heads with relations...
Generating knowledge graph...
Took 1.2147226333618164 seconds


### GPT-3 Based Commonsense Inference with custom relation

In [30]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.models.gpt3.zeroshot import GPT3Zeroshot
from kogito.core.relation import KnowledgeRelation, register_relation
import time, os

csi = CommonsenseInference()
csi.remove_processor("simple_relation_matcher")

def x_wishes_verbalizer(head, **kwargs):
   # index will be passed from the model
   # so that we can enumerate samples which helps with inference
   index = kwargs.get("index")
   index_txt = f"{index}" if index is not None else ""
   return f"Situation {index_txt}: {head}\nWishes: As a result, PersonX wishes"

X_WISHES = KnowledgeRelation("xWishes",
                             verbalizer=x_wishes_verbalizer,
                             prompt="How does this situation affect each character's wishes?")
register_relation(X_WISHES)

model = GPT3Zeroshot(api_key="", model_name="text-davinci-002")

sample_graph = KnowledgeGraph.from_csv("sample_graph2.tsv", sep="\t", header=None)

heads = ["PersonX makes a huge mistake", "PersonX sees PersonY's point"]

start = time.time()
kgraph = csi.infer(model=model,
                   heads=heads,
                   sample_graph=sample_graph,
                   model_args={"debug": True, "top_p": 0.5, "stop": "\n"})
end = time.time()

print(f"Took {end-start} seconds")
kgraph.to_jsonl("results/kgraph_gpt3_custom_relation.json")

Matching knowledge heads with relations...
Generating knowledge graph...
Took 1.7802231311798096 seconds


### Commonsense Knowledge Linking

In [31]:
from kogito.linkers.deberta import DebertaLinker
from kogito.core.knowledge import KnowledgeGraph

linker = DebertaLinker()
context = [
      "joey was pretending to drive his wife to work .",
      "the truth was that he was taking her on a trip .",
      "when they passed the road for her workplace , she asked what was up .",
      "that 's when he announced the trip detour plans .",
      "his wife was so thrilled and they really enjoyed their trip together ."
    ]
input_graph = KnowledgeGraph.from_csv("sample_linking_graph.csv", sep="|", header=None)
relevance_probs = linker.link(input_graph, context)
print(relevance_probs)

filtered_graph = linker.filter(input_graph, context)

print(filtered_graph)

Downloading: 100%|██████████| 2.46M/2.46M [00:00<00:00, 4.11MB/s]
Downloading: 100%|██████████| 56.0/56.0 [00:00<00:00, 18.7kB/s]
Downloading: 100%|██████████| 854/854 [00:00<00:00, 285kB/s]
Downloading: 100%|██████████| 1.74G/1.74G [02:24<00:00, 12.0MB/s] 


[[0.00024847043096087873], [6.804546865168959e-05], [0.9954416751861572]]
Knowledge(head="drive", relation="HasSubEvent", tails=['get into car'])


### Commonsense Inference with Filtering

In [32]:
from kogito.models.bart.comet import COMETBART
from kogito.inference import CommonsenseInference

model = COMETBART.from_pretrained()
csi = CommonsenseInference()

text = "PersonX wraps gifts"
kgraph = csi.infer(text, model)

kgraph.to_jsonl("kgraph_without_context.json")

context = ['hank had to wrap a lot of gifts for his family .', 
           'he ran out of wrapping paper with 4 gifts to go .',
           'he went to the kitchen and found shopping bags .', 
           'he cut up the bags to make sheets of paper .', 
           'he used the paper to wrap the last of the gifts .']

kgraph2 = csi.infer(text, model, context=context)
kgraph2.to_jsonl("kgraph_with_context.json")

Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


100%|██████████| 1/1 [00:12<00:00, 12.12s/it]


Extracting knowledge heads...
Matching knowledge heads with relations...
Generating knowledge graph...


100%|██████████| 1/1 [00:11<00:00, 11.95s/it]


Loading default Deberta linker for filtering...
Filtering knowledge graph based on the context...
